In [1]:
import os
import sys
sys.path.append('../../')
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import recs.path as path
from recs.util import readjson2dict, id2cat
from recs.cf_mf import CollaborativeFiltering
from recs.clf import Classifier
from recs.util import readjson2dict, save_model, save2json, recall_evaluate
from sklearn.model_selection import train_test_split

In [2]:
DIFF = True
TEST = True
merge_filename = "merge"
crm_filename = "crm"
if TEST:
    merge_filename += "_test"
if DIFF:
    merge_filename += "_diff"
    crm_filename += "_diff"

read_path = os.path.join(path.data_path, merge_filename + ".pkl")
trans_buy_path = os.path.join(path.data_path, "trans_buy.pkl")
trans_buy = pd.read_pickle(trans_buy_path)
merge = pd.read_pickle(read_path)


fund_info_path = os.path.join(path.data_path, "fund_info.pkl")
crm_path = os.path.join(path.data_path, crm_filename+".pkl")
# if TEST == False:
exist_funds_path = os.path.join(path.data_path, 'exist_funds_2021.json')
crm = pd.read_pickle(crm_path)
fund_info = pd.read_pickle(fund_info_path)

with open(exist_funds_path) as jf:
    exist_funds = json.load(jf)

i2idx = readjson2dict('fund_info_idx')
ef_idx = []
for i in exist_funds:
    try:
        ef_idx.append(i2idx[i])
    except:
        continue

In [3]:
merge.columns
merge = merge.replace(float('-inf'), 0)
# merge['foreign_fixed_deposit_pct'].value_counts()


In [4]:
merge[(merge.id_number == 13651)]
# merge

,id_number,fund_id,buy_date,yyyymm,y,local_foreign_total,local_total,local_demand_deposit,local_fixed_deposit,foreign_total,...,count_currency_EUR,count_currency_GBP,count_currency_HKD,count_currency_JPY,count_currency_NZD,count_currency_SEK,count_currency_SGD,count_currency_TWD,count_currency_USD,count_currency_ZAR
1,13651,748,20190709,201906,1,16.179238,16.069354,16.069354,0.0,13.916462,...,0,0,0,0,0,0,0,0,1,0
2,13651,2053,20170926,201708,1,15.772270,15.712601,15.712601,0.0,12.923644,...,0,0,0,0,0,0,0,0,1,0
3,13651,2339,20200121,201912,1,15.137757,13.303332,13.303332,0.0,14.963754,...,0,0,0,0,0,0,0,0,1,0
31952,13651,1512,20171121,201710,1,16.281395,15.652095,15.652095,0.0,15.520045,...,0,0,0,0,0,0,0,0,1,0
31953,13651,2648,20170821,201707,1,15.821352,15.765447,15.765447,0.0,12.909423,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239793,13651,1991,20190807,201907,0,16.372836,16.275772,16.275772,0.0,13.992309,...,1,0,0,0,0,0,0,0,0,0
239794,13651,178,20170602,201705,0,16.063885,15.610443,15.610443,0.0,15.054828,...,0,0,0,0,0,0,0,0,0,0
239795,13651,2384,20180607,201805,0,15.472325,15.349427,15.349427,0.0,13.315102,...,1,0,0,0,0,0,0,0,0,0
239796,13651,2233,20170703,201706,0,15.940613,15.892077,15.892077,0.0,12.891001,...,0,0,0,0,0,0,0,1,0,0


In [5]:
predict_result = merge[['id_number', 'fund_id']]
x = merge.drop(columns=['y', 'buy_date', 'yyyymm', 'id_number', 'fund_id'])
y = merge['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
y_train.value_counts()

0    338141
1    169068
Name: y, dtype: int64

In [64]:
c = Classifier(x_train, x_test, y_train, y_test)
c.do_gradient_boosting(lr=1.0, n_estimators=150, max_depth=3)

# model_path = os.path.join(path.model_path, 'model_diff_3')
# save_model(c.model)

Start Gradient Boosting Training...
      Iter       Train Loss   Remaining Time 
         1           1.1970            8.70m
         2           1.1387            8.70m
         3           1.0811            8.68m
         4           1.0702            8.55m
         5           1.0482            8.55m
         6           1.0218            8.56m
         7           1.0103            8.49m
         8           1.0011            8.41m
         9           0.9926            8.34m
        10           0.9832            8.24m
        20           0.9334            7.61m
        30           0.9136            7.01m
        40           0.9006            6.33m
        50           0.8878            5.71m
        60           0.8771            5.10m
        70           0.8675            4.50m
        80           0.8621            3.91m
        90           0.8580            3.34m
       100           0.8541            2.77m


In [29]:
c.check_score()
c.score_result

{'model_score': 0.8012318393403902,
 'accuracy_score': 0.8003122954504232,
 'f1_score': 0.6788834920675181,
 'precision_score': 0.7314713598600787,
 'recall_score': 0.6333498970682189}

In [30]:
x_test.columns

Index(['local_foreign_total', 'local_total', 'local_demand_deposit',
       'local_fixed_deposit', 'foreign_total', 'foreign_fixed_deposit',
       'foreign_demand_deposit', 'invest_type', 'age', 'monthly_trade_vol',
       'stock_inventory_val', 'KPI', 'local_foreign_total_diff',
       'local_total_diff', 'local_demand_deposit_diff',
       'local_fixed_deposit_diff', 'foreign_total_diff',
       'foreign_fixed_deposit_diff', 'foreign_demand_deposit_diff',
       'monthly_trade_vol_diff', 'stock_inventory_val_diff',
       'local_foreign_total_pct', 'local_total_pct',
       'local_demand_deposit_pct', 'local_fixed_deposit_pct',
       'foreign_total_pct', 'foreign_fixed_deposit_pct',
       'foreign_demand_deposit_pct', 'monthly_trade_vol_pct',
       'stock_inventory_val_pct', 'age_group', 'fund_type', 'local_or_foreign',
       'chosen', 'guaranteed', 'fund_type_web', 'invest_targets',
       'invest_region', 'currency', 'AUM_B', 'AUM_E', 'AUM_F', 'AUM_FT',
       'AUM_I', 'AUM_M'

In [31]:
fund_info.fund_id.isin(ef_idx).value_counts()

True     3266
False       1
Name: fund_id, dtype: int64

In [45]:
# best_sel = trans_buy[trans_buy.yyyymm  > 201900].fund_id.value_counts().tolist()[:100]
# len(best_sel)
# trans_buy[trans_buy.yyyymm  > 201900].fund_id.value_counts()[:100]

In [61]:
targets = crm[crm.yyyymm == 202012]
if TEST:
    targets = crm[crm.yyyymm == 201912]
# funds = fund_info[
#     (fund_info.fund_id.isin(ef_idx)) & 
#     (fund_info.fund_id.isin(best_sel)
# ]
funds = fund_info
     
targets['k'] = 0
funds['k'] = 0
print(len(targets))
print(len(funds))

69839
3267


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [62]:
pred_input = targets.merge(funds, how='outer')

In [63]:
pred_input.invest_type.value_counts()
pred_input.isnull().sum()
pred_input.fillna(0)
pred_input = pred_input.replace(np.inf, 0)
pred_input = pred_input.replace(float("-inf"), 0)
if DIFF:
    pred_input['local_demand_deposit_pct'].value_counts()

In [49]:
x_predict = pred_input[x_test.columns]
x_user_fund = pred_input[['id_number', 'fund_id']]

In [50]:
predict_prob = c.model.predict_proba(x_predict)
predict_prob

array([[0.90820828, 0.09179172],
       [0.90820828, 0.09179172],
       [0.97953733, 0.02046267],
       ...,
       [0.99593452, 0.00406548],
       [0.85846058, 0.14153942],
       [0.98344257, 0.01655743]])

In [38]:
predict_y = c.model.predict(x_predict)
predict_y

array([1, 0, 0, ..., 0, 0, 0])

In [53]:
numpy_prob = np.array(predict_prob)
print(predict_y[3593566])
print(len(predict_prob))
print(numpy_prob)
x_user_fund['prob'] = numpy_prob[:, 1]
print(x_user_fund)

1
20462827
[[0.90820828 0.09179172]
 [0.90820828 0.09179172]
 [0.97953733 0.02046267]
 ...
 [0.99593452 0.00406548]
 [0.85846058 0.14153942]
 [0.98344257 0.01655743]]
          id_number  fund_id      prob
0             38831        1  0.091792
1             38831        2  0.091792
2             38831        3  0.020463
3             38831        4  0.052241
4             38831        5  0.052241
...             ...      ...       ...
20462822      56616     1102  0.193106
20462823      56616     1274  0.095333
20462824      56616     1524  0.004065
20462825      56616     2296  0.141539
20462826      56616     2458  0.016557

[20462827 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [54]:
x_user_fund.fund_id.value_counts()

2830    69839
119     69839
99      69839
100     69839
101     69839
        ...  
313     69839
314     69839
316     69839
317     69839
1       69839
Name: fund_id, Length: 293, dtype: int64

In [55]:
print(x_user_fund[x_user_fund['id_number'] == 65136].sort_values(by=['prob'], ascending=False)[:10])
print("-----")
print(x_user_fund[x_user_fund['id_number'] == 0].sort_values(by=['prob'], ascending=False)[:10])

          id_number  fund_id      prob
14588750      65136      671  0.850191
14588554      65136     1123  0.850191
14588673      65136      255  0.825342
14588520      65136      170  0.825342
14588635      65136     1321  0.816140
14588505      65136      147  0.805587
14588734      65136      285  0.790115
14588539      65136      347  0.776319
14588672      65136      247  0.771641
14588607      65136      205  0.732651
-----
         id_number  fund_id      prob
9563031          0       41  0.818925
9563204          0      417  0.818925
9563205          0      418  0.818925
9563152          0      634  0.818925
9563006          0      548  0.818925
9563214          0      671  0.817583
9563018          0     1123  0.817583
9562953          0       20  0.815929
9563128          0       99  0.732864
9563071          0      205  0.726329


In [56]:
# raise SystemExit()
rec_result = {}
count = 0
for user in tqdm(x_user_fund.id_number.unique()):
    if TEST:
        ignore_item = trans_buy[(trans_buy.id_number == user) & (trans_buy.buy_date < 202000)].fund_id.unique().tolist()
    else:
        ignore_item = trans_buy[(trans_buy.id_number == user)].fund_id.unique().tolist()
    tmp = x_user_fund[x_user_fund.id_number==user]
    rec_fund = tmp[~tmp.fund_id.isin(ignore_item)].sort_values(by=['prob'], ascending=False).fund_id.astype(str).tolist()[:10]    
    rec_result[user] = rec_fund    

100%|██████████| 69839/69839 [12:18<00:00, 94.54it/s]


In [57]:
rec_result[49721]

['347', '1123', '671', '20', '170', '255', '205', '1321', '147', '247']

In [22]:
def backtest_ans(trans_df: pd.DataFrame, target_users: pd.Series, 
                start: int = 201200, end: int = 202000):
    DATE_KEY = "buy_date"
    USER_ID = "id_number"
    ITEM_ID = "fund_id"
    
    ans_df = trans_df[
        (trans_df[DATE_KEY] > start) &
        (trans_df[DATE_KEY] <= end)
    ]

    ans_users = ans_df[ans_df[USER_ID].isin(target_users)][USER_ID].unique().tolist()
    
    ans_dict = {}
    for u in tqdm(ans_users):
        ignore_items = trans_df[
            (trans_df[USER_ID] == u) &
            (trans_df[DATE_KEY] < start)
        ][ITEM_ID].unique().tolist()
        
        ans_dict[u] = ans_df[
            ~(ans_df[ITEM_ID].isin(ignore_items)) &
            (ans_df[USER_ID] == u)
        ][ITEM_ID].unique().astype(str).tolist()

    return ans_dict

if TEST:
    ans_dict = backtest_ans(trans_buy, targets.id_number, start= 20200000, end= 20210000)

100%|██████████| 18698/18698 [00:54<00:00, 340.74it/s]


In [ ]:
ans_dict[46487]

In [58]:
recall_evaluate(rec_result, ans_dict)

100%|██████████| 18698/18698 [00:00<00:00, 155875.04it/s]


{'recall@3': 0.002980385261456224,
 'recall@5': 0.004642450508723121,
 'recall@10': 0.00821765370125063}

In [25]:
print(rec_result[13627])
print(ans_dict[13627])

['347', '1321', '548', '418', '417', '634', '432', '1123', '671', '619']
['921', '1916', '2516', '2936', '1995', '3065']


In [26]:
dec_rec_result = {}
u2idx = readjson2dict("crm_idx")
i2idx = readjson2dict("fund_info_idx")

for user in tqdm(rec_result):
    dec_user = id2cat(u2idx, user)
    dec_funds = []
    for item in rec_result[user]:
        dec_funds.append(id2cat(i2idx, int(item)))
    dec_rec_result[dec_user] = dec_funds
#     break

 12%|█▏        | 8191/69839 [08:36<1:04:48, 15.85it/s]


KeyboardInterrupt: 

In [ ]:
# dec_rec_result

In [ ]:
model_save_path = os.path.join(path.export_path, "r_20211101_model_diff_2.json") 
with open(model_save_path, 'w') as jf:
    json.dump(dec_rec_result, jf)

In [ ]:
for i in dec_rec_result:
    print(dec_rec_result[i])
    break